In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob
!pip install cvlib
import cvlib as cv
from cvlib.object_detection import draw_bbox
import pandas as pd

In [2]:
# initial parameters
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

data = []
labels = []

# load image files from the dataset
BASE_DIR = "C://Users/User/Downloads/cnn/archive/UTKFace"
from tqdm.notebook import tqdm
from PIL import Image


In [17]:
age_labels = []
image_path = []
for filename in (os.listdir(BASE_DIR)):
    image_path.append(os.path.join(BASE_DIR, filename))

    temp = filename.split('_')
    age = int(temp[0])
    if 0<=age<3:
        age_labels.append('toddler')
    elif 3<=age<13:
        age_labels.append('child')
    elif 13<=age<20:
        age_labels.append('teen')
    elif 20<=age<30:
        age_labels.append('twenties')
    elif 30<=age<40:
        age_labels.append('thirties')
    elif 40<=age<50:
        age_labels.append('forties')
    elif 50<=age<60:
        age_labels.append('fifties')
    elif 60<=age<70:
        age_labels.append('sixties')
    else:
        age_labels.append('very old')


In [18]:
df = pd.DataFrame()
df['image_paths'], df['age'] = image_path, age_labels
df.head()

,image_paths,age
0,C://Users/User/Downloads/cnn/archive/UTKFace\1...,very old
1,C://Users/User/Downloads/cnn/archive/UTKFace\1...,very old
2,C://Users/User/Downloads/cnn/archive/UTKFace\1...,very old
3,C://Users/User/Downloads/cnn/archive/UTKFace\1...,very old
4,C://Users/User/Downloads/cnn/archive/UTKFace\1...,very old


In [5]:


# converting images to arrays and labelling the categories
for img in df.image_paths:

    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    age = int(img.split(os.path.sep)[-1][0]) # C:\Files\gender_dataset_face\woman\face_1162.jpg
    if 0<=age<3:
        label = 0#('toddler')
    elif 3<=age<13:
        label = 1#('child')
    elif 13<=age<20:
        label = 2#('teen')
    elif 20<=age<30:
        label = 3#('twenties')
    elif 30<=age<40:
        label = 4#('thirties')
    elif 40<=age<50:
        label = 5#('forties')
    elif 50<=age<60:
        label = 6#('fifties')
    elif 60<=age<70:
        label = 7#('sixties')
    else:
        label = 8#('very old')

        
    labels.append([label]) 
print(labels[5])

# pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels) 


[0]


In [6]:
# split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

trainY = to_categorical(trainY, num_classes=9) 
testY = to_categorical(testY, num_classes=9)

In [7]:
# define model
from tensorflow.keras.optimizers import Adam
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
    
    # The axis that should be normalized, after a Conv2D layer with data_format="channels_first", 
    # set axis=1 in BatchNormalization.

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model

# build model
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=9)

# compile the model
opt = Adam(learning_rate=lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the model
H = model.fit(trainX, trainY, batch_size=batch_size,
              validation_data=(testX, testY),
              steps_per_epoch=len(trainX) // batch_size,
              epochs=epochs, verbose=1)



296/296 [==============================] - 241s 800ms/step - loss: 0.2900 - accuracy: 0.5987 - val_loss: 0.1501 - val_accuracy: 0.6434


PermissionDeniedError: {{function_node __wrapped__MergeV2Checkpoints_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to create a directory: G:; Permission denied [Op:MergeV2Checkpoints]

In [8]:
model.save('G://age_detection.model')

INFO:tensorflow:Assets written to: G://age_detection.model/assets


INFO:tensorflow:Assets written to: G://age_detection.model/assets


1/1 [==============================] - 0s 73ms/step


1/1 [==============================] - 0s 77ms/step


1/1 [==============================] - 0s 28ms/step
